In [ ]:
# Install necessary packages
!pip install flask-ngrok
!pip install flask
!pip install tensorflow
!pip install pyngrok

# Import necessary libraries
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import os
from google.colab import drive
from pyngrok import ngrok



In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Authenticate ngrok
NGROK_AUTH_TOKEN = '2i7bVtYFj8iPsotfnvJzD5mhSCk_2zQ6zLXszjPntKjoNMkZD'  # Replace with your ngrok auth token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Create Flask app
app = Flask(__name__)

# Load the model
model_path = '/content/drive/MyDrive/Semester 4/Data Mining/model_trained.h5'
model = tf.keras.models.load_model(model_path)

# Define root route
@app.route('/', methods=['GET'])
def index():
    return "<h1>Hello World</h1>"

# Define prediction route
@app.route('/predict', methods=['POST'])
def upload():
    data = {"success": False}

    if 'file' not in request.files:
        data["file"] = "Tidak ada file"
        return jsonify(data)

    file = request.files['file']

    if file.filename == '':
        data["file"] = "Tidak ada file"
        return jsonify(data)

    # Secure the filename and save it
    filename = secure_filename(file.filename)
    filepath = os.path.join('/content/drive/MyDrive/Semester 4/Data Mining/Fast Food Classification V2/Test', filename)
    file.save(filepath)

    try:
        # Load and preprocess the image
        img = load_img(filepath, target_size=(224, 224))
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        x /= 255.0

        # Make a prediction
        predict = model.predict(x)

        # Interpret the prediction
        temp = 0
        label = 0
        hasil = []

        for y in range(10):
            presentase = predict[0][y] * 100
            hasil.append(presentase)
            if presentase > temp:
                temp = presentase
                label = y

        data["success"] = True
        data["label"] = label
        data["confidence"] = temp
        data["predictions"] = hasil

    except Exception as e:
        data["error"] = str(e)

    return jsonify(data)

if __name__ == '__main__':
    # Ensure the 'data_tes' directory exists
    if not os.path.exists('/content/drive/MyDrive/Semester 4/Data Mining/Fast Food Classification V2/Test'):
        os.makedirs('/content/drive/MyDrive/Semester 4/Data Mining/Fast Food Classification V2/Test')

    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f'Public URL: {public_url}')

    # Run the Flask app
    app.run()

Public URL: NgrokTunnel: "https://d723-35-227-51-23.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [08/Jul/2024 13:31:54] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 84ms/step


INFO:werkzeug:127.0.0.1 - - [08/Jul/2024 15:16:37] "POST /predict HTTP/1.1" 200 -
